# Requirement

### 1. Implement traditional one-versus-one and one-versus-rest task decomposition methods to solve a multi-class problem mentioned below.
### 2. Implement part-versus-part task decomposition method to solve the same multi-class problem.
### 3. Use two different kernel functions, namely linear and RBF, in all your classifiers.

******

#### Loading data and import svm

In [204]:
import numpy as np
import os, sys
sys.path.append('libsvm-3.22/python')
from svmutil import *

train_path = 'data/train.txt'
test_path = 'data/test.txt'

y, x = svm_read_problem(train_path) # training data
yt, xt = svm_read_problem(test_path) # test data

#### Implement traditional one-versus-one with two different kernel functions, namely linear and RBF

In [201]:
#-----------One-versus-rest needs k =12 classifiers

class OVR(object):
    def __init__(self, kernel=0):
        self.svm_dict = {}
        self.kernel = kernel
    def train(self,x,y):
        # decomposition
        for k in range(12):
            y_or = [(1 if i==k else -1) for i in y]
            m = svm_train(y_or, x, '-t %d -c 10 -b 1'% self.kernel)
            self.svm_dict[k] = m
            
    def predict(self,xt,yt):
        # combination
        prob_list = []
        for k in range(12):
            m = self.svm_dict[k]
            _, _, p_val = svm_predict(yt, xt, m, '-b 1 -q')
            prob_list.append(p_val)
        pl = np.array(prob_list)
        pl = pl[:,:,0]
        pred = np.argmax(pl, axis = 0)
        return pred

# ---- linear
ovr = OVR()
ovr.train(x,y)
pred = ovr.predict(xt,yt)
# evaluation
correct = np.sum(pred==yt)
acc = correct/float(len(yt))
print 'Accuracy with linear kernel: ', acc*100, '%'

# ---- RBF
ovr = OVR(kernel=2)
ovr.train(x,y)
pred = ovr.predict(xt,yt)
# evaluation
correct = np.sum(pred==yt)
acc = correct/float(len(yt)) 
print 'Accuracy with RBF kernel: ', acc*100, '%'

Accuracy with linear kernel:  58.2562747688 %
Accuracy with RBF kernel:  57.9260237781 %


#### Implement traditional one-versus-rest with two different kernel functions, namely linear and RBF

In [205]:
#-----------One-versus-one needs k*(k-1)/2 = 66 classifiers

class OVO(object):
    def __init__(self, kernel=0):
        self.svm_dict = {}
        self.kernel = kernel
        
    def dataO2O(self,x,y,i,j):
        xn, yn =[], []
        for ind,label in enumerate(y):
            if label==i:
                xn.append(x[ind])
                yn.append(1)
            if label==j:
                xn.append(x[ind])
                yn.append(-1)
        return xn,yn

    def train(self,x,y):
        # decomposition
        for i in range(12):
            for j in range(i+1,12):
                x_oo, y_oo = self.dataO2O(x,y,i,j)
                m = svm_train(y_oo, x_oo, '-t %d -c 10 -b 1'%self.kernel)
                self.svm_dict[i,j] = m
            
    def predict(self,xt,yt):
        # combination
        votes = np.zeros([1514, 12])
        for i in range(12):
            for j in range(i+1,12):
                m = self.svm_dict[i,j]
                _, _, p_val = svm_predict(yt, xt, m, '-b 1 -q')
                for n, val in enumerate(p_val):
                    if val[0]>val[1]:
                        votes[n,i] +=1
                    else:
                        votes[n,j] +=1
        pred = np.argmax(votes, axis=1)
        return pred

# ---- linear
ovo = OVO()
ovo.train(x,y)
pred = ovo.predict(xt,yt)
# evaluation
correct = np.sum(pred==yt)
acc = correct/float(len(yt))  
print 'Accuracy with linear kernel: ', acc*100, '%'

# ---- RBF
ovo = OVO(kernel=2)
ovo.train(x,y)
pred = ovo.predict(xt,yt)
# evaluation
correct = np.sum(pred==yt)
acc = correct/float(len(yt)) 
print 'Accuracy with RBF kernel: ', acc*100, '%'

Accuracy with linear kernel:  63.3421400264 %
Accuracy with RBF kernel:  60.8982826948 %


#### Implement traditional part-versus-part with two different kernel functions, namely linear and RBF

In [206]:
# --------- Part-versus-part with one-vs-rest needs k*(k-1)/2 = 66 classifiers

class PVP(object):
    def __init__(self, kernel=0):
        self.svm_dict = {}
        self.kernel = kernel
        
    def data2P(self,x,y):
        x_dict, y_dict = {}, {}
        xnp = np.array(x)
        ynp = np.array(y)
        len_opt = np.sum(ynp==1)
        len_neg = np.sum(ynp==-1)
        index_opt = np.array(np.where(ynp==1)).T
        index_neg = np.array(np.where(ynp==-1)).T
        index_opt = index_opt[:,0]
        index_neg = index_neg[:,0]
        np.random.shuffle(index_neg)
        for i in range(len_neg/len_opt):
            ind = np.hstack((index_opt, index_neg[i*len_opt:(i+1)*len_opt]))
            x_dict[i] = list(xnp[ind])
            y_dict[i] = list(ynp[ind])
        return x_dict, y_dict
    
    def train(self,x,y):
        # decomposition
        for k in range(12):
            y_or = [(1 if i==k else -1) for i in y]
            x_dict, y_dict = self.data2P(x,y_or)
            # train svms
            svm_part_dict = {}
            for p in range(len(y_dict)):
                m = svm_train(y_dict[p], x_dict[p], '-t 0 -c 10 -b 1')
                svm_part_dict[p] = m
            self.svm_dict[k] = svm_part_dict
            
    def predict(self,xt,yt):
        # combination
        prob_list = []
        for k in range(12):
            svm_part_dict = self.svm_dict[k]
            pred = np.zeros([1514,len(svm_part_dict)])
            for p in range(len(svm_part_dict)):
                m = svm_part_dict[p]
                _, _, p_val = svm_predict(yt, xt, m, '-b 1 -q')
                pred[:,p] = np.array(p_val)[:,0]
            pred = np.min(pred,axis=1)
            prob_list.append(pred)
            
        pl = np.array(prob_list)
        pl = pl[:,:]
        pred = np.argmax(pl, axis = 0)
        return pred
    
# ---- linear
pvp = PVP()
pvp.train(x,y)
pred = pvp.predict(xt,yt)
# evaluation
correct = np.sum(pred==yt)
acc = correct/float(len(yt))  
print 'Accuracy with linear kernel: ', acc*100, '%'

# ---- RBF
pvp = PVP(kernel=2)
pvp.train(x,y)
pred = pvp.predict(xt,yt)
# evaluation
correct = np.sum(pred==yt)
acc = correct/float(len(yt))  
print 'Accuracy with RBF kernel: ', acc*100, '%'

Accuracy with linear kernel:  60.5019815059 %
Accuracy with RBF kernel:  60.5680317041 %



### 4. Compare the advantages and disadvantages of these three task decomposition methods.
**One-vs-Rest:**   
Advantages:  
- k categories needs k svms.   
- can train svms parallelly.      

Disadvantages:   
- The Number of training data for each classifier is N.   
- The training data is strongly unbalanced, so the result are not so good.

**One-vs-One:**   
Advantages:     
- The Number of training data for each classifier is about 2N/k.   
- The training data is nearly balanced, so the result are better than One-vs-Rest.  

Disadvantages:   
- k categories needs kx(k-1)/2 svms, too many models  

**Part-vs-Part:**   
Advantages:  
- A large-scale two-class problem can be divided into a number of relatively smaller two-class problems
- A serious imbalance two-class problem can be divided into a number of balance two-class problems
- Massively parallel learning can be easily implemented

Disadvantages:   
- need preprocessing with training data.